In [17]:
# !pip install geopandas -q

In [18]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
# import geopandas as gpd
from shapely.geometry import Point

In [19]:
folder_path = '.'

In [20]:
years_list = list(range(2012, 2025))
# season = list(range(1,5))
# time_list = []
# type(time_list)
# for y in years_list:
#     for s in season:
#         startm = 3*(s-1) + 1
#         stopm = 3*s
#         time_list.append({'year': y, 'startm': startm, 'stopm': stopm})

In [21]:
# for item in time_list:
#     y = item['year']
#     startm = item['startm']
#     stopm = item['stopm']

### 空值檢查

In [32]:
for year in years_list:
    file_path = fr'{folder_path}/yeargroup-1022/{year}-1022.csv'

    data = pd.read_csv(file_path)

    # 計算 "經度" 和 "緯度" 欄位的空值數量
    missing_longitude = data['經度'].isnull().sum()
    missing_latitude = data['緯度'].isnull().sum()
    if missing_latitude != 0 | missing_longitude != 0:
        print(f'Year: {year}, Missing Longitude: {missing_longitude}, Missing Latitude: {missing_latitude}')


### 提醒!!!!!  <br> 修改的部分請加在def裡面即可，列印的部分修改output_folder就好其他盡量不要動 <br> 每個圖表儲存完後都一定要plt.close 已經先在列印的部分處理了

### 方法

In [23]:
def KdePlot(year):
    file_path = fr'{folder_path}/yeargroup-1022/{year}-1022.csv'
    data = pd.read_csv(file_path)

    plt.figure(figsize=(12, 10))

    kde_plot = sns.kdeplot(x=data['經度'], y=data['緯度'], fill=True)

    plt.title(f'KDE Plot of Values for {year}')
    plt.xlabel('Longitude (km)')
    plt.ylabel('Latitude (km)')
    plt.axis('equal')
    plt.colorbar(kde_plot.collections[0], label='Density')

In [24]:
def Valuepoints(year):
    file_path = fr'{folder_path}/yeargroup-1022/{year}-1022.csv'
    data = pd.read_csv(file_path)
    plt.figure(figsize=(12, 10))
    sns.scatterplot(x='經度', y='緯度', data=data, alpha=0.5)
    plt.title(f'Values for {year}')
    plt.xlabel('Longitude (km)')
    plt.ylabel('Latitude (km)')
    plt.grid(True)
    plt.axis('equal')

In [35]:
def KdeValue(year, bw_method):
    file_path = fr'{folder_path}/yeargroup-1022/{year}-1022.csv'
    data = pd.read_csv(file_path)

    # 假設這裡的 '經度' 和 '緯度' 已經轉換為公里
    longitude_km = data['經度']
    latitude_km = data['緯度']

    # 準備數據進行KDE計算
    data_points = np.vstack([longitude_km, latitude_km])

    # 使用 gaussian_kde 計算KDE
    kde = gaussian_kde(data_points, bw_method)

    # 直接計算原始數據點的KDE值
    kde_values = kde(data_points)

    return kde_values

### 列印圖表

In [26]:
# for year in years_list:
#     output_folder = f'{folder_path}KDEplot'
#     Kdefile_name = f'{output_folder}\\KDEplot{year}.png'
    
#     KdePlot(year)

#     plt.savefig(Kdefile_name, bbox_inches='tight')
#     plt.close()

In [27]:
# for year in years_list:
#     output_folder = f'{folder_path}Valuepoints'
#     Kdefile_name = f'{output_folder}\Valuepoints{year}.png'

#     Valuepoints(year)

#     plt.savefig(Kdefile_name, bbox_inches='tight')
#     plt.close()

In [28]:
file_path

'./yeargroup-1022/2024-1022.csv'

### 三種KDE

In [33]:
for year in years_list:

    #記得先做之前的dataprocess
    # file_path = fr'{folder_path}yeargroup-1022\{year}-1022.csv'
    # output_file_path = fr'{folder_path}yeargroupwithKDE\{year}-1022.csv'
    file_path = fr'{folder_path}/yeargroup-1022/{year}-1022.csv'
    output_file_path = fr'{folder_path}/yeargroupwithKDE-1022/{year}-1022.csv'

    
    Kde_df= pd.DataFrame()
    for bw in [0.5, 1, 1.5]:
        column_name = f'KDE_{bw}km'
        Kde_df[column_name] = KdeValue(year, bw)
    
    original_data = pd.read_csv(file_path)
    Kde_df = pd.concat([original_data, Kde_df], axis=1)

    Kde_df.to_csv(output_file_path, index=False, encoding='utf-8')

### 輸出只有編號+KDE*3

In [30]:
# output_file_path = fr'{folder_path}/yeargroupwithKDE-1022/kde_results.csv'

# # 創建一個空的 DataFrame 用於儲存所有結果
# all_results_df = pd.DataFrame()

# for year in years_list: 
#     file_path = fr'{folder_path}/yeargroupwithKDE-1022/{year}-1022.csv'
    
#     # 讀取 CSV 文件
#     data = pd.read_csv(file_path)
    
#     # 提取「編號」和三個 KDE 值
#     result_df = data[['編號', 'KDE_0.5km', 'KDE_1km', 'KDE_1.5km']]
    
#     # 將結果追加到總的 DataFrame 中
#     all_results_df = pd.concat([all_results_df, result_df], ignore_index=True)

# # 將所有結果保存到一個 CSV 文件
# all_results_df.to_csv(output_file_path, index=False)

In [34]:
import os
import pandas as pd

# 設定文件夾路徑
output_file_path = fr'{folder_path}/yeargroupwithKDE-1022/kde_results.csv'

# 創建一個空的 DataFrame 用於儲存所有結果
all_results_df = pd.DataFrame()

# 遍歷所有年份的文件
for year in years_list:
    file_path = fr'{folder_path}/yeargroupwithKDE-1022/{year}-1022.csv'
    
    # 讀取 CSV 文件
    data = pd.read_csv(file_path)
    
    # 將讀取到的數據合併到總的 DataFrame 中
    all_results_df = pd.concat([all_results_df, data], ignore_index=True)

# 只保留所需的「編號」和三個 KDE 值的列
all_results_df = all_results_df[['編號', 'KDE_0.5km', 'KDE_1km', 'KDE_1.5km']]

# 將所有結果保存到一個 CSV 文件
all_results_df.to_csv(output_file_path, index=False)


# 一種kde

In [37]:
def oneKdeValue(year):
    file_path = f'./yeargroup-1022/{year}-1022.csv'
    data = pd.read_csv(file_path)

    # 假設這裡的 '經度' 和 '緯度' 已經轉換為公里
    longitude_km = data['經度']
    latitude_km = data['緯度']

    # 準備數據進行KDE計算
    data_points = np.vstack([longitude_km, latitude_km])

    # 使用 gaussian_kde 計算KDE
    kde = gaussian_kde(data_points)

    # 直接計算原始數據點的KDE值
    kde_values = kde(data_points)

    return kde_values

In [39]:
for year in years_list:
    # 設置輸入文件路徑和輸出文件路徑
    file_path = f'./yeargroup-1022/{year}-1022.csv'
    output_file_path = f'./yeargroup-1022-onekde/{year}-1022.csv'

    # 打印開始處理的年份和讀取文件的訊息
    print(f'Reading data for year: {year} from {file_path}...')
    
    # 讀取 CSV 文件
    data = pd.read_csv(file_path)
    
    # 打印 KDE 計算的訊息
    print(f'Calculating KDE for year: {year}...')
    data['KDE'] = oneKdeValue(year)
    
    # 打印保存文件的訊息
    print(f'Saving updated data with KDE for year: {year} to {output_file_path}...')
    data.to_csv(output_file_path, index=False)
    
    # 打印完成處理的訊息
    print(f'Finished processing for year: {year}\n')


Reading data for year: 2012 from ./yeargroup-1022/2012-1022.csv...
Calculating KDE for year: 2012...
Saving updated data with KDE for year: 2012 to ./yeargroup-1022-onekde/2012-1022.csv...
Finished processing for year: 2012

Reading data for year: 2013 from ./yeargroup-1022/2013-1022.csv...
Calculating KDE for year: 2013...
Saving updated data with KDE for year: 2013 to ./yeargroup-1022-onekde/2013-1022.csv...
Finished processing for year: 2013

Reading data for year: 2014 from ./yeargroup-1022/2014-1022.csv...
Calculating KDE for year: 2014...
Saving updated data with KDE for year: 2014 to ./yeargroup-1022-onekde/2014-1022.csv...
Finished processing for year: 2014

Reading data for year: 2015 from ./yeargroup-1022/2015-1022.csv...
Calculating KDE for year: 2015...
Saving updated data with KDE for year: 2015 to ./yeargroup-1022-onekde/2015-1022.csv...
Finished processing for year: 2015

Reading data for year: 2016 from ./yeargroup-1022/2016-1022.csv...
Calculating KDE for year: 2016...